In [2]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import matplotlib.pyplot as plt

from CRUDMilestone import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Initialize shelter object and log in as aacuser
username = 'aacuser'
password = 'SNHU1234'
shelter = AnimalShelter(username, password)


# class read method returns a list object
df = pd.DataFrame.from_records(shelter.read({}))

# Drop the id column
# If this is not done, the data will not be usable by the dashboard
df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Open Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Create Widgets
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    # Create filter radio items
    html.Div(
        dcc.RadioItems(['Water Rescue', 'Wilderness Rescue', 'Individual Tracking', 'Show All'], 
                        'Show All', inline=True, id='filter-type')
    ),
    html.Hr(),
    # Create Data Table
    dt.DataTable(
        id='datatable-id',
        columns=[
            {'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Format table to be more readable
        page_size = 15,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        row_selectable = 'single'
    ),
    html.Hr(),
    # Unique identifier code
    html.Blockquote("Created by Kayleigh Kinsey for CS-340 : 10/12/23"),
    html.Hr(),
    #This sets up the dashboard so that the pie chart and the geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Hr(),
    # Display Grazioso Salvare Logo
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
])


#############################################
# Interaction Between Components / Controller
#############################################



@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### Filter interactive data table with MongoDB queries
        if filter_type == 'Water Rescue':
            df = pd.DataFrame.from_records(shelter.read({'animal_type' : 'Dog', 
                                                         'breed' : {'$in':['Labrador Retriever Mix', 
                                                                           'Chesapeake Bay Retriever', 
                                                                           'Newfoundland']}, 
                                                         'sex_upon_outcome' : 'Intact Female', 
                                                         'age_upon_outcome_in_weeks' : {'$gte':26}, 
                                                         'age_upon_outcome_in_weeks' : {'$lte':156}}))
        elif filter_type == 'Wilderness Rescue':
            df = pd.DataFrame.from_records(shelter.read({'animal_type' : 'Dog', 
                                                         'breed' : {'$in':['German Shepherd', 
                                                                           'Alaskan Malamute', 
                                                                           'Old EnglishSheepdog', 
                                                                           'Siberian Husky',
                                                                           'Rottweiler']}, 
                                                         'sex_upon_outcome' : 'Intact Male', 
                                                         'age_upon_outcome_in_weeks' : {'$gte':26}, 
                                                         'age_upon_outcome_in_weeks' : {'$lte':156}}))
        elif filter_type == 'Individual Tracking':
            df = pd.DataFrame.from_records(shelter.read({'animal_type' : 'Dog', 
                                                         'breed' : {'$in':['Doberman Pinscher', 
                                                                           'GermanShepherd', 
                                                                           'Golden Retriever',
                                                                           'Bloodhound', 
                                                                           'Rottweiler']}, 
                                                         'sex_upon_outcome' : 'Intact Male', 
                                                         'age_upon_outcome_in_weeks' : {'$gte':20}, 
                                                         'age_upon_outcome_in_weeks' : {'$lte':300}}))
        elif filter_type == 'Show All':
            df = pd.DataFrame.from_records(shelter.read({}))
        
        # Drop the id column
        # If this is not done, the data will not be usable by the dashboard
        df.drop(columns=['_id'],inplace=True)
        df.drop_duplicates(subset='rec_num', inplace=True)
        # Create columns and data
        columns=[{'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in df.columns]
        data=df.to_dict('records')
       
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
### Update colors for selected columns
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'data')])
def update_graphs(data): 
### Update pie chart data
    breedDict = {}
    for item in data:
        if item["breed"] not in breedDict:
            breedDict[item["breed"]] = 1
        else:
            breedDict[item["breed"]] += 1    
    
    pieNames = []
    pieValues = []

    for key, value in breedDict.items():
        pieNames.append(key)
        pieValues.append(value)
    
    return [
        dcc.Graph(figure = px.pie(values=pieValues, names=pieNames))
    ]

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('datatable-id', "derived_viewport_selected_rows")])
def update_map(viewData, index):
### Update the geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None or type(index) is not int:
       row = 0
    else: 
       row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id='base-layer-id'),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
                  dl.Tooltip(dff.iloc[row,4]),
                  dl.Popup( children=
                    #html.H1('Animal Name'),
                    html.H1(dff.iloc[row,9])
                  )
              ])
           ])
    ]


app.run_server(debug=True)

Dash app running on http://127.0.0.1:12081/
